In [3]:
#load libraries
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re, os

In [4]:
raw_data = pd.read_csv("/Users/albk/Documents/Code/Real_Projects/basketball_league_webapp/data/Carleton|Laurentian|2022-11-13.csv")

In [5]:
player_event_df = raw_data.copy()

In [6]:
player_event_df['V-event'] = player_event_df['V-event'].astype(str)

In [7]:
#clean data
players_list = ['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN','SERAPH,REGINALD JEAN','LATIFF,WAZIR','SHEPHARD,GRANT','RILEY,BEN', 'SAMAHA,GABRAEL','OKADO,MARJOK','BAILEY,ELLIOT','MEYER,EVAN','KNIGHT,JACOB','DOLMAN,LUKA','KATCHUNGA,JORDAN']
event_list = ['made layup','missed layup','Assist','Turnover','defensive rebound','enters the game','goes to the bench','missed 3-pt. jump shot','Foul','Steal','made free throw','missed free throw','made jump shot','made 3-pt. jump shot','missed jump shot','offensive rebound']

### Simply building a dataframe from main one and getting kinds of event with respect to their player

In [8]:
pattern = "([A-Z]+,[A-Z]+.?[A-Z]*)"
for index, row in player_event_df.iterrows():
    player = re.search(pattern, row["V-event"])
    if player:
        player = player[0].strip()
        player_event_df.loc[index, "player"] = player
    
    for event in event_list:
        if event in row["V-event"]:
            player_event_df.loc[index, "event"] = event

In [9]:
player_event_df.loc[player_event_df["player"] == "VREEKEN,CONNOR"]

,Unnamed: 0,Time,Home,H-event,Score,V-event,Visitor,player,event
1,1,09:58,NaN,NaN,0 - 2,"VREEKEN,CONNOR made layup",Carleton,"VREEKEN,CONNOR",made layup
20,20,07:43,NaN,NaN,2 - 4,"VREEKEN,CONNOR missed 3-pt. jump shot",Carleton,"VREEKEN,CONNOR",missed 3-pt. jump shot
35,35,06:08,NaN,NaN,4 - 9,"Steal by VREEKEN,CONNOR",Carleton,"VREEKEN,CONNOR",Steal
53,53,06:00,NaN,NaN,5 - 13,"VREEKEN,CONNOR goes to the bench",Carleton,"VREEKEN,CONNOR",goes to the bench
67,67,04:41,NaN,NaN,5 - 16,"VREEKEN,CONNOR enters the game",Carleton,"VREEKEN,CONNOR",enters the game
77,77,03:49,NaN,NaN,5 - 18,"VREEKEN,CONNOR made layup",Carleton,"VREEKEN,CONNOR",made layup
84,84,03:49,NaN,NaN,6 - 18,"VREEKEN,CONNOR missed jump shot",Carleton,"VREEKEN,CONNOR",missed jump shot
110,110,01:42,NaN,NaN,6 - 25,"VREEKEN,CONNOR made layup",Carleton,"VREEKEN,CONNOR",made layup
113,113,01:26,NaN,NaN,6 - 25,"VREEKEN,CONNOR defensive rebound",Carleton,"VREEKEN,CONNOR",defensive rebound
127,127,00:39,NaN,NaN,8 - 25,"VREEKEN,CONNOR goes to the bench",Carleton,"VREEKEN,CONNOR",goes to the bench


### Calculating in-game time of each player on each quarter and events count for each player also

In [84]:
quarter_indices = list(player_event_df[player_event_df['Time'].str.contains('Quarter')].index)
quarter_indices.append(len(player_event_df) - 1)
quarter_dict = {"player":players_list, "time1":[], "score1":[], 
                "time2":[], "score2":[], "time3":[], "score3":[],
                "time4":[], "score4":[]}
quarter = 1

in_players = []
starter = ['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN',
           'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 
           'SHEPHARD,GRANT']

not_changed_list = ["not_changed" for n in range(len(players_list))]
event_num_dict = {"player":[]}
time_dict = {"player":players_list, 
             "seconds":list(np.zeros(len(players_list))),
             "points_conceded":list(np.zeros(len(players_list))),
             "timecache":not_changed_list.copy(),
             "scorecache":not_changed_list.copy()}
             

for ind, row in player_event_df.iterrows():
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
    # calculating in-game time of each player
    # -------------------------------------------
    # each quarter end calculations
    if ind in quarter_indices:
        if ind != 0:
            for player in in_players:
                player_ind = time_dict["player"].index(player)
                cached_time = time_dict["timecache"][player_ind]
                cached_score = time_dict["scorecache"][player_ind]
                if cached_time == "not_changed":
                    enter_time = datetime.strptime("10:00", "%M:%S")
                    enter_score_index = quarter_indices[quarter - 1] + 1
                    enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
                else:
                    enter_time = datetime.strptime(cached_time, "%M:%S")
                    enter_score = cached_score
                
                seconds = enter_time - datetime.strptime("00:00", "%M:%S")
                seconds = seconds.total_seconds()

                points = int(player_event_df.iloc[ind - 1]["Score"].split("-")[0]) - enter_score

                time_dict["seconds"][player_ind] += seconds
                time_dict["points_conceded"][player_ind] += points

            quarter += 1
            quarter_dict["player"] = time_dict["player"]
            quarter_dict[f"time{quarter_indices.index(ind)}"] = time_dict["seconds"]
            quarter_dict[f"score{quarter_indices.index(ind)}"] = time_dict["points_conceded"]
            time_dict["timecache"] = not_changed_list.copy()
            time_dict["scorecache"] = not_changed_list.copy()
            time_dict["seconds"] = list(np.zeros(len(players_list)))
            time_dict["points_conceded"] = list(np.zeros(len(players_list)))

        in_players = starter.copy()
        continue
    # -------------------------------------------
    # iterating rows calculation
    if not pd.isna(row["player"]) and row["player"] != np.nan:
        player_index = time_dict["player"].index(row["player"])
        cached_time = time_dict["timecache"][player_index]
        cached_score = time_dict["scorecache"][player_index]
        if "goes to the bench" in row["event"]:
            in_players.remove(row["player"])
            if cached_time == "not_changed":
                enter_time = datetime.strptime("10:00", "%M:%S")
                enter_score_index = quarter_indices[quarter - 1] + 1
                enter_score = int(player_event_df.iloc[enter_score_index]["Score"].split("-")[0])
            else:
                enter_time = datetime.strptime(cached_time, "%M:%S")
                enter_score = cached_score
            
            seconds = enter_time - datetime.strptime(row["Time"], "%M:%S")
            seconds = seconds.total_seconds()

            points = int(row["Score"].split("-")[0]) - enter_score

            time_dict["seconds"][player_index] += seconds
            time_dict["points_conceded"][player_index] += points

        elif "enters the game" in row["event"]:
            in_players.append(row["player"])
            time_dict["timecache"][player_index] = row["Time"]
            time_dict["scorecache"][player_index] = int(row["Score"].split("-")[0])
    # -------------------------------------------
# ------------------------------------------------------------------------------------------------------------------------------------------------------------
        # filling event_num_dict for meaesuring how many times each event occured
        if row["event"] not in event_num_dict:
            event_num_dict[row["event"]] = list(np.zeros(len(event_num_dict["player"])))
        
        if row["player"] not in event_num_dict["player"]:
            event_num_dict["player"].append(row["player"])
            for key in event_num_dict:
                if key != "player":
                    event_num_dict[key].append(0)
        
        player_index = event_num_dict["player"].index(row["player"])
        event_num_dict[row["event"]][player_index] += 1
# ------------------------------------------------------------------------------------------------------------------------------------------------------------

time_score_df = pd.DataFrame(quarter_dict)

columns_ls = []
for i in range(1, 5):
    for sub in ["time", "score"]:
        columns_ls.append((f"quarter{i}", sub))

time_score_df.columns = pd.MultiIndex.from_tuples([("player", "player")] + columns_ls)

In [85]:
event_num_df = pd.DataFrame(event_num_dict)
time_sum_ls = [(f"quarter{i}", "time") for i in range(1, 5)]
score_sum_ls = [(f"quarter{i}", "score") for i in range(1, 5)]
time_score_df[("total", "time")] = time_score_df[time_sum_ls].sum(axis=1)
time_score_df[("total", "score")] = time_score_df[score_sum_ls].sum(axis=1)

In [86]:
time_score_df

player quarter1       quarter2       quarter3        \
                  player     time score     time score     time score   
0         VREEKEN,CONNOR    482.0   8.0    306.0  13.0    258.0   5.0   
1        WARNHOLTZ,AIDEN    443.0   7.0    452.0  13.0    600.0  17.0   
2   SERAPH,REGINALD JEAN    232.0   1.0    157.0  14.0    107.0   3.0   
3           LATIFF,WAZIR    479.0   5.0    329.0  13.0    385.0   9.0   
4         SHEPHARD,GRANT    153.0   1.0    292.0  17.0    174.0   3.0   
5              RILEY,BEN    204.0   5.0    408.0   6.0    234.0   7.0   
6         SAMAHA,GABRAEL    329.0   7.0    134.0   4.0    385.0   9.0   
7           OKADO,MARJOK    200.0   3.0    362.0  12.0    108.0   5.0   
8          BAILEY,ELLIOT    118.0   0.0    329.0  13.0      0.0   0.0   
9             MEYER,EVAN    118.0   0.0    174.0   2.0      0.0   0.0   
10          KNIGHT,JACOB    153.0   1.0     57.0   8.0      0.0   0.0   
11           DOLMAN,LUKA     89.0   2.0      0.0   0.0      0.0   0.0   
12      KATCHUNGA,JORDAN      0.0   0.0      0.0   0.0      0.0   0.0   

   quarter4         total        
       time score    time score  
0     448.0   6.0  1494.0  32.0  
1     600.0  16.0  2095.0  53.0  
2       0.0   0.0   496.0  18.0  
3     600.0  16.0  1793.0  43.0  
4     536.0  12.0  1155.0  33.0  
5     548.0  15.0  1394.0  33.0  
6       0.0   0.0   848.0  20.0  
7       0.0   0.0   670.0  20.0  
8     268.0  15.0   715.0  28.0  
9       0.0   0.0   292.0   2.0  
10      0.0   0.0   210.0   9.0  
11      0.0   0.0    89.0   2.0  
12      0.0   0.0     0.0   0.0

In [14]:
event_num_df

,player,made layup,Assist,Turnover,defensive rebound,enters the game,goes to the bench,missed 3-pt. jump shot,Foul,made 3-pt. jump shot,Steal,made free throw,missed free throw,made jump shot,missed jump shot,offensive rebound,missed layup
0,"VREEKEN,CONNOR",4,0.0,1.0,5.0,6.0,9.0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,1.0,2.0,2.0
1,"WARNHOLTZ,AIDEN",1,12.0,2.0,10.0,2.0,3.0,5.0,4.0,3.0,0.0,6.0,1.0,0.0,3.0,2.0,5.0
2,"SERAPH,REGINALD JEAN",0,1.0,1.0,0.0,4.0,8.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,"LATIFF,WAZIR",0,6.0,2.0,5.0,4.0,5.0,6.0,1.0,2.0,3.0,2.0,0.0,0.0,0.0,0.0,5.0
4,"SHEPHARD,GRANT",5,0.0,1.0,8.0,5.0,7.0,0.0,2.0,0.0,0.0,9.0,6.0,0.0,0.0,6.0,5.0
5,"RILEY,BEN",0,0.0,1.0,1.0,8.0,4.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
6,"SAMAHA,GABRAEL",2,0.0,0.0,1.0,6.0,6.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0
7,"OKADO,MARJOK",0,1.0,2.0,3.0,7.0,5.0,1.0,5.0,1.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0
8,"BAILEY,ELLIOT",4,1.0,1.0,0.0,8.0,5.0,2.0,5.0,1.0,2.0,1.0,4.0,0.0,1.0,2.0,1.0
9,"MEYER,EVAN",0,0.0,0.0,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


off possession = made and miss

def possession = Turnover, defensive rebound,	Foul, Steal

efficiency = point, rebound, assist, steal, block, -missed free throw, -turnover, -points conceded

What type of game was it? (Pre-season, Regular Season, or Playoffs)

In [93]:
scoring_values = {'made layup': 2, 'made free throw': 1, 'made jump shot': 2, 'made 3-pt. jump shot': 3}
final_columns = ['Player Name', 'PtScored', 'ptsconceded', 
                #  'off possession', 'def possession', 
                "efficiency",
                'minutes', 'home/visitor', 'opponent', 'date', 'game_type']
player_final_table = pd.DataFrame(columns=[final_columns])
for index, row in event_num_df.iterrows():
    
    points_scored = 0
    for key in scoring_values:
        score_value = scoring_values[key]
        points_scored += event_num_df.loc[index, key] * score_value

    points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])

    efficiency = sum([points_scored, row["offensive rebound"], row["defensive rebound"], row["Assist"], row["Steal"]]) - \
    sum([row["missed free throw"], row["Turnover"], points_conceded])
    efficiency = float(efficiency)

    minutes = time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "time")] / 60
    minutes = "{:.2f}".format(minutes.to_list()[0])
    hv_df = player_event_df.loc[player_event_df["player"] == row["player"]].iloc[1]
    if pd.isna(hv_df['Home']) == False:
        hv = "Home"
    else:
        hv = "Visitor"
    
    opponent_df = player_event_df.loc[pd.isna(player_event_df[hv]) == True]
    opponent = opponent_df.iloc[1]["Home"] if hv == "Visitor" else opponent_df.iloc[1]["Visitor"]

    new_row = {"Player Name":[row["player"]], "PtScored":[points_scored],
               "ptsconceded":[points_conceded], "efficiency":[efficiency], "minutes":minutes, 
               "home/visitor":hv, "opponent":opponent}
    
    new_df = pd.DataFrame(new_row)
    player_final_table = pd.concat([player_final_table, new_df], ignore_index=True, axis=0)

player_final_table = player_final_table.reindex(columns=final_columns)

/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_2920/1449936464.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])
/var/folders/hx/k_zr_cdj5l348hsyxvym51kr0000gn/T/ipykernel_2920/1449936464.py:14: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  points_conceded = float(time_score_df.loc[time_score_df[("player", "player")] == row["player"]][("total", "score")])


In [94]:
player_final_table

,Player Name,PtScored,ptsconceded,efficiency,minutes,home/visitor,opponent,date,game_type
0,"VREEKEN,CONNOR",11.0,32.0,-13.0,24.90,Visitor,Laurentian,NaN,NaN
1,"WARNHOLTZ,AIDEN",17.0,53.0,-15.0,34.92,Visitor,Laurentian,NaN,NaN
2,"SERAPH,REGINALD JEAN",0.0,18.0,-18.0,8.27,Visitor,Laurentian,NaN,NaN
3,"LATIFF,WAZIR",8.0,43.0,-23.0,29.88,Visitor,Laurentian,NaN,NaN
4,"SHEPHARD,GRANT",19.0,33.0,-7.0,19.25,Visitor,Laurentian,NaN,NaN
5,"RILEY,BEN",2.0,33.0,-31.0,23.23,Visitor,Laurentian,NaN,NaN
6,"SAMAHA,GABRAEL",6.0,20.0,-13.0,14.13,Visitor,Laurentian,NaN,NaN
7,"OKADO,MARJOK",5.0,20.0,-11.0,11.17,Visitor,Laurentian,NaN,NaN
8,"BAILEY,ELLIOT",12.0,28.0,-16.0,11.92,Visitor,Laurentian,NaN,NaN
9,"MEYER,EVAN",0.0,2.0,-1.0,4.87,Visitor,Laurentian,NaN,NaN


## Let's go to lineup

In [111]:
quarter_indices = list(player_event_df[player_event_df['Time'].str.contains('Quarter')].index)
quarter_indices.append(len(player_event_df) - 1)
lineup_quarter_dict = {"lineup":players_list, "quarter1":[], "quarter2":[], 
                "quarter3":[], "quarter4":[]}
quarter = 1

starter_lineup = ['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 
                  'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 
                  'SHEPHARD,GRANT']

lineup_time_dict = {"lineup":[], "seconds":[], "cache":[]}
lineup_event_dict = {}
in_lineup = []

for ind, row in player_event_df.iterrows():
    if ind in quarter_indices:
        if ind != 0:
            quarter += 1

        in_lineup = starter_lineup.copy()
        lineup_time_dict["lineup"].append(in_lineup)
        continue
    
    print(lineup_time_dict["lineup"][-1])
    print("!", in_lineup)

    if not pd.isna(row["player"]):
        if "goes to the bench" in row["event"]:
            in_lineup.remove(row["player"])
        elif "enters the game" in row["event"]:
            in_lineup.append(row["player"])

        last_lineup = lineup_time_dict["lineup"][-1]
        if sorted(in_lineup) != sorted(last_lineup) and len(in_lineup) == 5:
            lineup_time_dict["lineup"].append(in_lineup)
            lineup_time_dict["cache"].append(row["Time"])

['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 'SHEPHARD,GRANT']
! ['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 'SHEPHARD,GRANT']
['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 'SHEPHARD,GRANT']
! ['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 'SHEPHARD,GRANT']
['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 'SHEPHARD,GRANT']
! ['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 'SHEPHARD,GRANT']
['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 'SHEPHARD,GRANT']
! ['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 'SHEPHARD,GRANT']
['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 'SHEPHARD,GRANT']
! ['VREEKEN,CONNOR', 'WARNHOLTZ,AIDEN', 'SERAPH,REGINALD JEAN', 'LATIFF,WAZIR', 'SHEPHARD,GRANT']
['VREEKEN,CONNOR', 'WARNHOLTZ,

In [101]:
ls = [1, 2, 3]
ls.remove(1)

print(ls)

[2, 3]


In [37]:
import pandas as pd

# Create a sample dataframe
data = {'A': [1, 2, 3], 'B': [4, 5, 6], "C":[4, 5, 6]}
df = pd.DataFrame(data)

# Create multi-level columns
df.columns = pd.MultiIndex.from_tuples([('Column 1', 'time'), ('Column 1', 'score'), ('Column 2', 'time')])

# Display the dataframe
print(df)

  Column 1       Column 2
      time score     time
0        1     4        4
1        2     5        5
2        3     6        6
